In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# extract variable names from header in DeepSolar data
deepsolar_header = pd.read_csv('../deepsolar_tract.csv', encoding='ISO-8859-1', nrows=0).columns.tolist()

In [3]:
# Look at the variables
print("Number of variables in DeepSolar database:", len(deepsolar_header), "\n")
for var in deepsolar_header:
    print(var)

Number of variables in DeepSolar database: 169 

Unnamed: 0
tile_count
solar_system_count
total_panel_area
fips
average_household_income
county
education_bachelor
education_college
education_doctoral
education_high_school_graduate
education_less_than_high_school
education_master
education_population
education_professional_school
employed
gini_index
heating_fuel_coal_coke
heating_fuel_electricity
heating_fuel_fuel_oil_kerosene
heating_fuel_gas
heating_fuel_housing_unit_count
heating_fuel_none
heating_fuel_other
heating_fuel_solar
land_area
per_capita_income
population
population_density
poverty_family_below_poverty_level
poverty_family_count
race_asian
race_black_africa
race_indian_alaska
race_islander
race_other
race_two_more
race_white
state
total_area
unemployed
water_area
education_less_than_high_school_rate
education_high_school_graduate_rate
education_college_rate
education_bachelor_rate
education_master_rate
education_professional_school_rate
education_doctoral_rate
race_white_ra

##### Code for choosing variables to use

There are two ways to do this:  
1) To start with the full set of variables and delete unused variables, run the code block above to load the header row from the DeepSolar database, then run/edit the code blocks below to remove unused variables.   
2) To use only a smaller set of named variables, use and run the single code block below to define the `variables` list.



Some decision boundaries for excluding variables:  

* Variables that are used for calculating other variables should not be included.
* If a proportion of a variable is available, a count is not needed.  
* If two variables are highly positively correlated with each other, one is removed.  Cutoff: 0.90 to 1.00 positive correlation.  
* If two variables are higly negatively correlated with each other, one is removed.  Cutoff:  -0.90 to -1.00 negative correlation.  Exception: retained both frost days and air temperature as two temperature measures.
* Variables that show minimal correlation with the outcome variable are removed.  Cutoff: $\leq$ 1.0% correlation.



In [3]:
# This code initializes a variable dictionary that by default includes all variables
# Variables to be included when loading the database have a dictionary value of 1
variables_dict = {}
for var in deepsolar_header:
    variables_dict[var] = 1

In [4]:
# This code defines which variables to NOT include by giving them a dictionary value that is not 1

# NOTE: 'Unnamed: 0' variable is used as an index in the pandas dataframe
# NOTE: fips numbers will be used for indentifying census tracts
# NOTE:  'number_of_solar_system_per_household' is outcome variable currently being used for analysis

# These variables are unused outcome variables from DeepSolar
variables_dict['tile_count'] = 'Unused outcome variable: total number of tiles in census tract'
variables_dict['solar_system_count'] = 'Unused outcome variable: Total number of solar systems in census tract'
variables_dict['total_panel_area'] = 'Unused outcome variable: Total surface of solar tiles in the census tract'
variables_dict['solar_panel_area_divided_by_area'] = 'Unused outcome variable: \
    total residential panel area divided by total area'
variables_dict['solar_panel_area_per_capita'] = 'Unused outcome variable: \
    total residential panel area divided by population'
variables_dict['tile_count_residential'] = 'Unused outcome variable: total residential tiles in census tract'
variables_dict['tile_count_nonresidential'] = 'Unused outcome variable: \
    total nonresidential tiles in census tract'
variables_dict['solar_system_count_residential'] = 'Unused outcome variable: \
    total residential solar systems in census tract'
variables_dict['solar_system_count_nonresidential'] = 'Unused outcome variable: \
    total nonresidential solar systems in census tract'
variables_dict['total_panel_area_residential'] = 'Unused outcome variable: \
    total residential panel area in census tract'
variables_dict['total_panel_area_nonresidential'] = 'Unused outcome variable: \
    total nonresidential panel area in census tract'

In [5]:
# These variables are unused geographic units
variables_dict['county'] = 'Geographic unit not used for analysis' 
variables_dict['land_area'] = 'Geographic unit not used for analysis'
variables_dict['total_area'] = 'Geographic unit not used for analysis'
variables_dict['water_area'] = 'Geographic unit not used for analysis'
# If longitude is needed for mapping in new data by geographic coordinates, comment out the following line
variables_dict['lon'] = 'Geographic unit not used for analysis'
# Consider trying models with and without state data 
# variables_dict['state'] = 'Geographic unit not used for analysis'

In [6]:
# These variables are used to calculate other variables and/or
# are counts represented in other variables as proportions
variables_dict['education_bachelor'] = 'Proportion recorded in another variable: education_bachelor_rate'
variables_dict['education_college'] = 'Proportion recorded in another variable: education_college_rate'
variables_dict['education_doctoral'] = 'Proportion recorded in another variable: education_doctoral_rate'
variables_dict['education_high_school_graduate'] = 'Proportion recorded in another variable: \
    education_high_school_graduate_rate'
variables_dict['education_less_than_high_school'] = 'Proportion recorded in another variable: \
    education_less_than_high_school_rate'
variables_dict['education_master'] = 'Proportion recorded in another variable: education_master_rate'
variables_dict['education_population'] = 'Used for calculating eduation proportions'
variables_dict['education_professional_school'] = 'Proportion recorded in another variable: \
    education_professional_school_rate'
variables_dict['employed'] = 'Proportion recorded in another variable: employ_rate'
variables_dict['heating_fuel_coal_coke'] = 'Proportion recorded in another variable: heating_fuel_coal_coke_rate'
variables_dict['heating_fuel_electricity'] = 'Proportion recorded in another variable: heating_fuel_electricity_rate'
variables_dict['heating_fuel_fuel_oil_kerosene'] = 'Proportion recorded in another variable: \
    heating_fuel_fuel_oil_kerosene_rate'
variables_dict['heating_fuel_gas'] = 'Proportion recorded in another variable: heating_fuel_gas_rate'
variables_dict['heating_fuel_housing_unit_count'] = 'Used for calculating heating proportions'
variables_dict['heating_fuel_none'] = 'Proportion recorded in another variable: heating_fuel_none_rate'
variables_dict['heating_fuel_other'] = 'Proportion recorded in another variable: heating_fuel_other_rate'
variables_dict['heating_fuel_solar'] = 'Proportion recorded in another variable: heating_fuel_solar_rate'
variables_dict['population'] = 'Used for calculating population density'
variables_dict['poverty_family_below_poverty_level'] = 'Proportion recorded in another variable: \
    poverty_family_below_poverty_level_rate'
variables_dict['poverty_family_count'] = 'Used for calculating poverty level rate'
variables_dict['race_asian'] = 'Proportion recorded in another variable: race_asian_rate'
variables_dict['race_black_africa'] = 'Proportion recorded in another variable: race_black_africa_rate'
variables_dict['race_indian_alaska'] = 'Proportion recorded in another variable: race_indian_alaska_rate'
variables_dict['race_islander'] = 'Proportion recorded in another variable: race_islander_rate'
variables_dict['race_other'] = 'Proportion recorded in another variable: race_other_rate'
variables_dict['race_two_more'] = 'Proportion recorded in another variable: race_two_more_rate'
variables_dict['race_white'] = 'Proportion recorded in another variable: race_white_rate'
variables_dict['unemployed'] = 'Proportion recorded in another variable: employ_rate'
variables_dict['household_count'] = 'Used for calculating other household proportions'
variables_dict['housing_unit_count'] = 'Used for calculating other housing unit proportions'
variables_dict['housing_unit_occupied_count'] = 'Used for calculating other occupancy rates'

In [7]:
# These variables are highly correlated with another variable 
variables_dict['electricity_price_commercial'] = 'Positively correlated with electricity_price_residential'
variables_dict['electricity_price_overall'] = 'Positively correlated with electricity_price_residential'
variables_dict['avg_electricity_retail_rate'] = 'Positively correlated with electricity_price_residential'
variables_dict['electricity_consume_industrial'] = 'Positively correlated with electricity_consume_total'
variables_dict['incentive_count_nonresidential'] = 'Positively correlated with incentive_count_residential'
variables_dict['heating_design_temperature'] = 'Positively correlated with air_temperature'
variables_dict['earth_temperature'] = 'Positively correlated with air_temperature'
variables_dict['heating_degree_days'] = 'Positively correlated with frost_days'
variables_dict['cooling_degree_days'] = 'Positively correlated with air_temperature'
variables_dict['voting_2012_dem_percentage'] = 'Positively correlated with voting_2016_dem_percentage'
variables_dict['voting_2012_gop_percentage'] = 'Positively correlated with voting_2016_gop_percentage'
variables_dict['average_household_income'] = 'Positively correlated with median_household_income and \
    per_capita_income'
variables_dict['atmospheric_pressure'] = 'Negatively correlated with elevation'
# If latitude is needed for mapping in new data by geographic coordinates, comment out the following line
variables_dict['lat'] = 'Negatively correlated with air_temperature'

In [8]:
# These variables are minimally correlated with the outcome variable:
variables_dict['travel_time_20_29_rate'] = 'Correlation with outcome variable: 0.004877743253901644'
variables_dict['age_75_84_rate'] = 'Correlation with outcome variable: 0.0037689775466753623'
variables_dict['transportation_walk_rate'] = 'Correlation with outcome variable: 0.0035459997452238425'
variables_dict['age_15_17_rate'] = 'Correlation with outcome variable: 0.0035085350984517564'
variables_dict['occupation_arts_rate'] = 'Correlation with outcome variable: 0.003117778141984627'
variables_dict['age_45_54_rate'] = 'Correlation with outcome variable: 0.0027423095259330229'
variables_dict['transportation_car_alone_rate'] = 'Correlation with outcome variable: 0.0021423621883868232'
variables_dict['wind_speed'] = 'Correlation with outcome variable: 0.00014976839853103464'
variables_dict['age_35_44_rate'] = 'Correlation with outcome variable: -0.0087885091220027758'
variables_dict['transportation_carpool_rate'] = 'Correlation with outcome variable: -0.0088712537526224686'
variables_dict['race_white_rate'] = 'Correlation with outcome variable: -0.0089163384931202159'
variables_dict['earth_temperature_amplitude'] = 'Correlation with outcome variable: -0.009753996923045842'
variables_dict['age_10_14_rate'] = 'Correlation with outcome variable: -0.00975482960293215'

In [9]:
# Consider not using these for residential anaysis
variables_dict['electricity_price_transportation'] = 'Not used for residential analysis'
variables_dict['electricity_consume_commercial'] = 'Not used for residential analysis'
variables_dict['incentive_nonresidential_state_level'] = 'Not used for residential analysis'

In [10]:
# Convert dictionary to list to use when loading pandas dataframe
variables = []
for key,val in variables_dict.items():
    if val==1:
        variables.append(key)
# Look at the revised set of variables
print("Number of variables in DeepSolar database:", len(variables), "\n")
for var in variables:
    print(var)

Number of variables in DeepSolar database: 92 

Unnamed: 0
fips
gini_index
per_capita_income
population_density
state
education_less_than_high_school_rate
education_high_school_graduate_rate
education_college_rate
education_bachelor_rate
education_master_rate
education_professional_school_rate
education_doctoral_rate
race_black_africa_rate
race_indian_alaska_rate
race_asian_rate
race_islander_rate
race_other_rate
race_two_more_rate
employ_rate
poverty_family_below_poverty_level_rate
heating_fuel_gas_rate
heating_fuel_electricity_rate
heating_fuel_fuel_oil_kerosene_rate
heating_fuel_coal_coke_rate
heating_fuel_solar_rate
heating_fuel_other_rate
heating_fuel_none_rate
median_household_income
electricity_price_residential
electricity_price_industrial
electricity_consume_residential
electricity_consume_total
average_household_size
housing_unit_median_value
housing_unit_median_gross_rent
elevation
cooling_design_temperature
frost_days
air_temperature
relative_humidity
daily_solar_radiation


In [14]:
# To choose only a small set of named variables, this is just an example
# IMPORTANT: To use the original index column that is in the DeepSolar database,
# always include the 'Unnamed: 0' variable first in the variable list
variables = ['Unnamed: 0', 'number_of_solar_system_per_household', 'daily_solar_radiation']

In [14]:
# load data into pandas dataframe
deepsolar = pd.read_csv('../deepsolar_tract.csv', encoding='ISO-8859-1', index_col=0, usecols=variables)

In [6]:
deepsolar.head()

,fips,average_household_income,gini_index,per_capita_income,population,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,...,diversity,number_of_solar_system_per_household,incentive_count_residential,incentive_residential_state_level,net_metering,feedin_tariff,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,27145011200,70352.789869,0.3490,26999.0,6577,44.52005,0.073879,0.386324,0.371592,0.125110,...,0.046960,0.000000,34,11,34,0,25,12,0,9.46
1,27145011301,61727.085202,0.4074,20951.0,8020,482.64430,0.062836,0.217096,0.405887,0.190773,...,0.145934,0.006726,34,11,34,0,25,12,0,9.46
2,27145011302,71496.886583,0.3926,28021.0,6971,186.16730,0.064509,0.343973,0.325670,0.190625,...,0.009150,0.001112,34,11,34,0,25,12,0,9.46
3,27145011304,86840.152755,0.3949,29275.0,6328,84.13610,0.066840,0.316840,0.322917,0.185185,...,0.187334,0.000000,34,11,34,0,25,12,0,9.46
4,27145011400,89135.315597,0.4463,32770.0,5264,119.63230,0.048686,0.293309,0.392473,0.195341,...,0.090766,0.002087,34,11,34,0,25,12,0,9.46


## Missing data - percentage per variable

In [15]:
# Calculate, sort, print proportion of missing data for each variable
missing_data = deepsolar.isnull().mean(axis=0).sort_values(ascending=False)
with pd.option_context('display.max_rows', None):
    print(missing_data)

daily_solar_radiation                      0.079987
elevation                                  0.079987
air_temperature                            0.079987
frost_days                                 0.079987
cooling_design_temperature                 0.079987
relative_humidity                          0.079987
housing_unit_median_gross_rent             0.027241
housing_unit_median_value                  0.026111
mortgage_with_rate                         0.015661
dropout_16_19_inschool_rate                0.014627
median_household_income                    0.013193
gini_index                                 0.011677
travel_time_average                        0.011677
poverty_family_below_poverty_level_rate    0.011346
average_household_size                     0.011139
heating_fuel_gas_rate                      0.010629
heating_fuel_coal_coke_rate                0.010629
heating_fuel_electricity_rate              0.010629
heating_fuel_fuel_oil_kerosene_rate        0.010629
heating_fuel

##### Summary of missing data

Note: missing data percentages above are calculated on the reduced data set after removing variables that are not expected to be used in analysis.  

* 8 variables have about 8% missing data, and these are all temperature data.  It's possible that there are 8% of census tracts that do not have a NOAA weather station in those census tracts. Also some NOAA weather stations collect less detailed weather data than others. This is a significant proportion of missing data that could impact results.  Suggestion for replacing missing values in these variables:  Use latitude and longitude of the census tracts with missing data to map to the weather data of the census tract with the nearest lat/lon values, or perhaps to an average of the weather data of geographically nearby census tracts within a specific radius.  


* 2 variables have almost 3% missing data: housing_unit_median_gross_rent and housing_unit_median_value. It would be interesting to see if the missing data in these variables coincides with missing data (or zero values) of the housing unit count variables. Possibly it isn't an absence of housing unit data but an absense of housing units.

Thought: for exploring residential data, we might want to eliminate census tracts that have zero housing units. 

* Remaining variables have less than 1.5% missing data, which would not be expected to dramatically impact results, so the method used to replace missing data (if replaced at all).

## Unique values per variable

Useful for identifying categorical variables (even if the values in the variable are numeric)

In [11]:
# Calculate, sort, print number of unique values for each variable
unique_values = deepsolar.nunique().sort_values(ascending=True)
with pd.option_context('display.max_rows', None):
    print(unique_values[:10])

voting_2016_dem_win                   2
voting_2012_dem_win                   2
feedin_tariff                         4
cooperate_tax                         4
rebate                                8
sales_tax                            12
incentive_residential_state_level    16
property_tax                         20
net_metering                         20
incentive_count_residential          22
dtype: int64
